In [125]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

In [126]:
class MultiScaleStockDataset(Dataset):
    """多模尺度数据集"""
    def __init__(self, scale_factors, seq_length=24, pred_length=4, data_path='./'):
        self.standardScaler = StandardScaler()
        self.scale_factors = scale_factors
        self.seq_length = seq_length
        self.pred_length = pred_length
        self.data_path = data_path
        self.data = {
            scale: self._generate_synthetic_data(scale) for scale in self.scale_factors
        }
    
    def _generate_synthetic_data(self, scale):
        """获取合成数据"""
        data = pd.read_csv(self.data_path + f'train-{scale}.csv', 
                           usecols=['date','open', 'high', 'low', 'close'],
                           parse_dates=['date'], index_col='date')
        return self.standardScaler.fit_transform(data)
    
    def _align_scales(self, idx):
        """三线性插值对齐时间轴"""
        aligned = {}
        base_idx = idx // 8  # 对齐到日线级别
        for scale in self.scale_factors:
            ratio = {'30min':8, '1hour':4, '4hour':1, '1day':1/24}[scale]
            start = int(base_idx * ratio)
            aligned[scale] = self.data[scale][start:start+self.seq_length]
        return aligned
    
    def __getitem__(self, idx):
        inputs = self._align_scales(idx)
         # 确保填充后的维度一致性
        for k,v in inputs.items():
            assert v.shape[1] == 4, f"特征维度错误: {k}的维度为{v.shape[1]}"

        # target = self.data['30min'][idx:idx+self.pred_length, 0]  # 预测未来价格
         # 统一所有尺度特征维度为4
        aligned = {k: np.pad(v, ((0,0),(0,1))) if v.shape[1]==3 else v 
              for k,v in inputs.items()}
        target = self.data['30min'][idx+self.pred_length-1, 0]  # 从[4]变为标量
        # 双重验证
        assert isinstance(target, np.float64), "目标值类型错误"
        target_tensor = torch.tensor(target, dtype=torch.float32)
        return {k:torch.FloatTensor(v) for k,v in aligned.items()}, target_tensor
    def __len__(self):
        return len(self.data[self.scale_factors[0]]) - self.pred_length - self.seq_length

class ScaleAwareExpert(nn.Module):
    """专家"""
    def __init__(self, input_dim, scale_type):
        super().__init__()
        self.scale_type = scale_type

        if '30min' in scale_type or '1h' in scale_type:
            """高频专家: CNN + BiLSTM """
            self.conv = nn.Conv1d(input_dim,
                                  out_channels=16,
                                  kernel_size=3, 
                                  padding='same', 
                                  )
            self.lstm = nn.LSTM(16,32, num_layers=2, batch_first=True, bidirectional=True)
            self.proj = nn.Linear(64,32)
        else:
            """低频专家: LSTM """
            self.proj_in = nn.Linear(input_dim, 4)  # 3维→4维
            self.encoder_layer = nn.TransformerEncoderLayer(d_model=4, nhead=2, dim_feedforward=64)
            self.transformer = nn.TransformerEncoder(self.encoder_layer, num_layers=3)
            self.proj = nn.Linear(4, 32)
        self.predictor = nn.Linear(32, 1)

    def forward(self, x):
        print(f"输入维度: {x.shape}")  # 应显示(batch, seq_len, 4)
        if '30min' in self.scale_type or '1h' in self.scale_type:
            x = self.conv(x.permute(0,2,1)).permute(0,2,1)
            x, _ = self.lstm(x)
            x = x[:, -1, :]  # 取最后一个时间步 [batch, 4]
            x = self.proj(x)
        else:
            x = self.proj_in(x)
            x = self.transformer(x)
            x = self.proj(x[:,-1,:]) # 只取最后一个时间步的输出
        return self.predictor(x)
class MoE(nn.Module):
    """混合专家网络"""
    def __init__(self, input_dim=4, experts=['30min','1hour','4hour','1day'], hidden_dim=64, output_dim=1):
        super().__init__()
        self.experts = nn.ModuleDict({
            scale: ScaleAwareExpert(input_dim, scale) for scale in experts
        })
        self.gate = nn.Sequential(
            nn.Linear(input_dim * len(experts), hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, len(experts)),
            nn.Softmax(dim=-1)
        )

    def forward(self, inputs):
        expert_outputs = {}
        for scale, expert in self.experts.items():
            expert_outputs[scale] = expert(inputs[scale])
        # 动态门控
        gate_input = torch.cat([v for v in inputs.values()], dim=-1)
        weights = self.gate(gate_input.mean(dim=1))  # (batch, num_experts)
        
        # 加权融合
        combined = sum(weights[:, i] * expert_outputs[scale] 
                      for i, scale in enumerate(self.experts.keys()))
        return combined, expert_outputs

In [127]:
# ==================== 训练逻辑 ====================
def hybrid_loss(pred, target, expert_outs, alpha=0.7):
    pred = pred.squeeze(-1)  # (32,1) → (32)
    mse = nn.MSELoss()
    main_loss = mse(pred, target)
    
    # 趋势一致性约束
    trends = torch.stack([torch.sign(out.detach()) for out in expert_outs.values()])
    consistency = torch.mean(torch.prod(trends, dim=0))
    
    return alpha*main_loss + (1-alpha)*(1 - consistency)
def train():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    dataset = MultiScaleStockDataset(data_path='../data/', scale_factors=['30min', '1hour', '4hour', '1day'])
    
    print(f"数据特征维度: {dataset.data['30min'].shape[1]}")
    loader = DataLoader(dataset, batch_size=32, shuffle=True)
    
    model = MoE(input_dim=4).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
    
    for epoch in range(50):
        model.train()
        for batch_idx, (inputs, target) in enumerate(loader):
            inputs = {k:v.to(device) for k,v in inputs.items()}
            target = target.to(device)
            
            pred, expert_outs = model(inputs)
            loss = hybrid_loss(pred, target, expert_outs)
            
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
        scheduler.step()
        print(f'Epoch {epoch} Loss: {loss.item():.4f}')

In [128]:
# ==================== 评估指标 ==================== 
def sharpe_ratio(returns, risk_free=0.0):
    excess_returns = returns - risk_free
    return excess_returns.mean() / excess_returns.std()

def max_drawdown(returns):
    cumulative = returns.cumsum()
    peak = cumulative.expanding(min_periods=1).max()
    drawdown = (peak - cumulative).max()
    return drawdown

In [129]:
def test_conv_layer():
    # 模拟输入 (batch=2, seq=24, features=4)
    test_input = torch.randn(2, 24, 4)
    expert = ScaleAwareExpert(input_dim=4, scale_type='30min')
    
    # 添加维度跟踪
    print("=== 维度跟踪 ===")
    x = expert.conv(test_input.permute(0,2,1)).permute(0,2,1)
    print(f"卷积后维度: {x.shape}")  # 应显示[2,24,16]
    
    x, _ = expert.lstm(x)
    print(f"LSTM后维度: {x.shape}")  # 应显示[2,24,64]
    
    x = expert.proj(x)
    print(f"Proj后维度: {x.shape}")  # 应显示[2,24,32]
    
    output = expert(test_input)
    print(f"最终输出维度: {output.shape}")  # 应显示[2,1]
    # 前向传播检查
    try:
        output = expert(test_input)
        print(f"测试通过，输出维度: {output.shape}")
    except Exception as e:
        print(f"测试失败: {str(e)}")

test_conv_layer()

=== 维度跟踪 ===
卷积后维度: torch.Size([2, 24, 16])
LSTM后维度: torch.Size([2, 24, 64])
Proj后维度: torch.Size([2, 24, 32])
输入维度: torch.Size([2, 24, 4])
最终输出维度: torch.Size([2, 1])
输入维度: torch.Size([2, 24, 4])
测试通过，输出维度: torch.Size([2, 1])


In [130]:
# 测试低频专家
def test_low_freq_expert():
    test_input = torch.randn(2, 24, 4)
    expert = ScaleAwareExpert(input_dim=4, scale_type='4hour')
    
    print("=== 低频专家维度跟踪 ===")
    x = expert.proj_in(test_input)
    print(f"投影后维度: {x.shape}")  # [2,24,4]
    
    x = x.permute(1,0,2)
    print(f"转置后维度: {x.shape}")  # [24,2,4]
    
    x = expert.transformer(x)
    print(f"Transformer后维度: {x.shape}")  # [24,2,4]
    
    output = expert(test_input)
    print(f"最终输出维度: {output.shape}")  # [2,1]

test_low_freq_expert()

=== 低频专家维度跟踪 ===
投影后维度: torch.Size([2, 24, 4])
转置后维度: torch.Size([24, 2, 4])
Transformer后维度: torch.Size([24, 2, 4])
输入维度: torch.Size([2, 24, 4])
最终输出维度: torch.Size([2, 1])


In [ ]:
train()